A good place to look would be the emails betwen Kenneth Lay, Jeffrey Skilling, and Andrew Fastow

In [15]:
import nltk,glob,os,sys,time,IPython,csv
from email.parser import Parser
from os.path import expanduser
from datetime import datetime
from datetime import date
home = expanduser("~")
sound_file = '/Users/Colin/Documents/horse.wav'

In [2]:
#convert file to email object
def toEmail(file):
    zoo = open(file, 'r',errors='ignore') #had issues with the open function so I forced the open to ignore errors
    moo = zoo.read() #read in file
    email = Parser().parsestr(moo)
    return(email)

In [25]:
#count the number of emails a person has sent to each person(source: his/her directory)
def sentEmails(name):
    first = name.lower().split(' ')[0]
    last = name.lower().split(' ')[1]
    fName = last+'-'+list(first)[0]
    directory = home+'/midterm/data/enron/maildir'
    dirty = os.path.join(directory,fName)
    mailAdd = first+'.'+last+'@enron.com'
    zoo = dict()
    folds = glob.glob(dirty+'/*')
    for each in folds:
        fids = glob.glob(each+'/*.')
        for n in fids:
            email = toEmail(n)
            if email['from'] == mailAdd:
                sent = email['to']
                if sent is not None:
                    if ',' in sent:
                        sent = sent.split(',')
                        sent = [s.strip() for s in sent]
                        for x in sent:
                            if x in zoo:
                                zoo[x] += 1
                            else:
                                zoo[x] = 1
                    else:
                        if sent in zoo:
                            zoo[sent] += 1
                        else:
                            zoo[sent] = 1
    return zoo

In [24]:
#count the number of emails a person has received from each person (source: his/her directory)
def receivedByName(name):
    first = name.lower().split(' ')[0]
    last = name.lower().split(' ')[1]
    fName = last+'-'+list(first)[0]
    directory = home+'/midterm/data/enron/maildir'
    dirty = os.path.join(directory,fName)
    mailAdd = first+'.'+last+'@enron.com'
    zoo = dict()
    folds = glob.glob(dirty+'/*')
    for each in folds:
        fids = glob.glob(each+'/*.')
        for n in fids:
            email = toEmail(n)
            sent = email['from']
            ID = email['message-id']
            #print(ID)
            to = email['to']
            tos = []
            if to is not None:
                if ',' in to:
                    tos = to.split(',')
                    tos = [s.strip() for s in to]
                if (mailAdd in tos) or (mailAdd == to):
                    if sent is not None:
                        if ',' in sent:
                            sent = sent.split(',')
                            sent = [s.strip() for s in sent]
                            for name in sent:
                                if name in zoo:
                                    if ID not in zoo[name]:
                                        zoo[name][ID] = email
                                else:
                                    zoo[name]={}
                                    zoo[name][ID] = email
                        else:
                            if sent in zoo:
                                if ID not in zoo[sent]:
                                    zoo[sent][ID] = email
                            else:
                                zoo[sent]={}
                                zoo[sent][ID] = email
    return zoo

In [23]:
#create a list of emails sent by an individual, sorted by receiver (source: his/her directory)
def sentByName(name):
    first = name.lower().split(' ')[0]
    last = name.lower().split(' ')[1]
    fName = last+'-'+list(first)[0]
    directory = home+'/midterm/data/enron/maildir'
    dirty = os.path.join(directory,fName)
    mailAdd = first+'.'+last+'@enron.com'
    zoo = dict()
    folds = glob.glob(dirty+'/*')
    for each in folds:
        fids = glob.glob(each+'/*.')
        for n in fids:
            email = toEmail(n)
            if email['from'] == mailAdd:
                sent = email['to']
                ID = email['message-id']
                #print(ID)
                if sent is not None:
                    if ',' in sent:
                        sent = sent.split(',')
                        sent = [s.strip() for s in sent]
                        for name in sent:
                            if name in zoo:
                                if ID not in zoo[name]:
                                    zoo[name][ID] = email
                            else:
                                zoo[name]={}
                                zoo[name][ID] = email
                    else:
                        if sent in zoo:
                            if ID not in zoo[sent]:
                                zoo[sent][ID] = email
                        else:
                            zoo[sent]={}
                            zoo[sent][ID] = email
    return zoo

In [22]:
#create a list of emails received by an individual, sorted by sender (source: his/her directory)
def receivedByName(name):
    first = name.lower().split(' ')[0]
    last = name.lower().split(' ')[1]
    fName = last+'-'+list(first)[0]
    directory = home+'/midterm/data/enron/maildir'
    dirty = os.path.join(directory,fName)
    mailAdd = first+'.'+last+'@enron.com'
    zoo = dict()
    folds = glob.glob(dirty+'/*')
    for each in folds:
        fids = glob.glob(each+'/*.')
        for n in fids:
            email = toEmail(n)
            to = email['to']
            tos = []
            if to is not None:
                if ',' in to:
                    tos = to.split(',')
                    tos = [s.strip() for s in to]
                if (mailAdd in tos) or (mailAdd == to):
                    sent = email['from']
                    ID = email['message-id']
                    #print(ID)
                    if sent is not None:
                        if ',' in sent:
                            sent = sent.split(',')
                            sent = [s.strip() for s in sent]
                            for name in sent:
                                if name in zoo:
                                    if ID not in zoo[name]:
                                        zoo[name][ID] = email
                                else:
                                    zoo[name]={}
                                    zoo[name][ID] = email
                        else:
                            if sent in zoo:
                                if ID not in zoo[sent]:
                                    zoo[sent][ID] = email
                            else:
                                zoo[sent]={}
                                zoo[sent][ID] = email
    return zoo

In [28]:
#create a list of emails between two people (source: their folders)
def emailsBetweenEnron(a,b):
    aFirst = a.lower().split(' ')[0]
    aLast = a.lower().split(' ')[1]
    bFirst = b.lower().split(' ')[0]
    bLast = b.lower().split(' ')[1]
    aMail = aFirst+'.'+aLast+'@enron.com'
    bMail = bFirst+'.'+bLast+'@enron.com'
    aList = sentByName(a) #sent by a
    bList = receivedByName(b) #received by b
    cList = receivedByName(a) #received by a
    dList = sentByName(b) #sent by b
    finList = {}
    if bMail in aList.keys():
        finList.update(aList[bMail])
    if aMail in bList.keys():
        finList.update(bList[aMail])
    if bMail in cList.keys():
        finList.update(cList[bMail])
    if aMail in dList.keys():
        finList.update(dList[aMail])
    print(len(finList),'total between',a,'and',b)
    return finList

In [47]:
kList = sentEmails('Kenneth Lay')
jList = sentEmails('Jeff Skilling')
#aList = sentEmails('Andrew Fastow') Fastow's emails not in file system

In [48]:
numList = sorted(kList.items(), key=lambda a: a[1], reverse = True)
someList = []
for each in numList:
    someList.append([each[0],each[1]])
#export restaurant data to CSV file
with open('ana_3/KLay_Email_Frequency.csv', 'w', newline='') as rest:
    writer = csv.writer(rest, delimiter=',')
    writer.writerow(["Person", "Number of Emails"])
    writer.writerows(someList)

In [49]:
numList = sorted(jList.items(), key=lambda a: a[1], reverse = True)
someList = []
for each in numList:
    someList.append([each[0],each[1]])
#export restaurant data to CSV file
with open('ana_3/JSkilling_Email_Frequency.csv', 'w', newline='') as rest:
    writer = csv.writer(rest, delimiter=',')
    writer.writerow(["Person", "Number of Emails"])
    writer.writerows(someList)

In [29]:
kaList = emailsBetweenEnron('Kenneth Lay','Andrew Fastow')
ajList = emailsBetweenEnron('Andrew Fastow','Jeff Skilling')
kjList = emailsBetweenEnron('Kenneth Lay','Jeff Skilling')

2 total between Kenneth Lay and Andrew Fastow
5 total between Andrew Fastow and Jeff Skilling
0 total between Kenneth Lay and Jeff Skilling


In [50]:
someList = []
for each in kaList:
    email = kaList[each]
    subject = email['subject']
    date = email['date']
    body = email.get_payload()
    someList.append([fro,to,date,subject,each,body])
#export restaurant data to CSV file
with open('ana_3/Lay_Fastow_Emails.csv', 'w', newline='') as rest:
    writer = csv.writer(rest, delimiter=',')
    writer.writerow(["Date", "Subject", "ID", "Body"])
    writer.writerows(someList)

In [51]:
someList = []
for each in ajList:
    email = ajList[each]
    subject = email['subject']
    date = email['date']
    body = email.get_payload()
    someList.append([fro,to,date,subject,each,body])
#export restaurant data to CSV file
with open('ana_3/Fastow_Slikking_Emails.csv', 'w', newline='') as rest:
    writer = csv.writer(rest, delimiter=',')
    t
    writer.writerows(someList)